In [ ]:
from multiprocessing import process
import sys
import math
import numpy as np
import itertools
from scipy.special import gamma, factorial
from matplotlib import pyplot as plt
from itertools import product
import random
from osgeo import gdal
import datetime
import time
import multiprocessing
from functools import partial
import numba
from numba import jit
#from line_profiler import LineProfiler


In [ ]:
# Function to filter through the slope to only use values greater than or equal to the critical slope value for failure. Outputs the index of where the values are.
@jit(nopython = True)
def slopeFilter(slopeArray,slope_value):
    rows = []
    cols = []
    for i in range(0,np.shape(slopeArray)[0]):
        for j in range(0,np.shape(slopeArray)[1]):
            if slopeArray[i][j] >= slope_value:
                rows.append(i)
                cols.append(j)
    return np.array(rows),np.array(cols)

# Function for finding landslide placements. Finds where landslide can fit on the boolean slope array by slicing slope array. If all values are true (>= critical value),
# then the row and column of the upper left hand corner are saved. Then randomly choses the row and column where the landslide should be placed.
# Outputs a list of data: [row, columns, landslide length, landslide depth]
@jit(nopython = True)
def landslideMasker(lslength,lsdepth,lsfreq,slopeDEM,slopeRows,slopeCols):
    row = []
    col = []
    lsdata = []
    for i in range (0,len(slopeRows)):
        if slopeRows[i]+lslength <= np.shape(slopeDEM)[0] and slopeCols[i]+lslength <= np.shape(slopeDEM)[1]:
            landslideMatrix = slopeDEM[slopeRows[i]:slopeRows[i]+lslength, slopeCols[i]:slopeCols[i]+lslength]
            if np.all(landslideMatrix):
                row.append(slopeRows[i])
                col.append(slopeCols[i])
    if len(row) > 0:
        row_index = np.arange(len(row))#for randomising which index to use for landslide
        rand_index = np.random.choice(row_index,size=lsfreq)
        #print(lslength,rand_index)

        for lsindex in rand_index:
            lsdata.append([row[lsindex],col[lsindex],lslength,lsdepth])
    else:
        print("couldn't find a landslide spot")
    return lsdata


# Function to make sure landslides are not overlapping. For each landslide checks to see if any values are FALSE.
# If FALSE values then places landslide in 'failed_data' list. If all values are TRUE places
# in 'passed_data' list then changes values to FALSE so next lanslide cannot be placed there
def landslideChecker(slope_array_boolean,landslideData):
    passed_data = []
    failed_data = []

    for currentLandslide in landslideData:
        originPoint = [int(currentLandslide[0]),int(currentLandslide[1])]
        lslength = int(currentLandslide[2])
        lsdepth = currentLandslide[3]
        working_mask = slope_array_boolean[originPoint[0]:originPoint[0]+lslength,originPoint[1]:originPoint[1]+lslength]
        if not np.all(working_mask):
            failed_data.append(np.array([originPoint[0],originPoint[1],lslength,lsdepth]))
            continue
        #if np.all(working_mask):
        slope_array_boolean[originPoint[0]:originPoint[0]+lslength,originPoint[1]:originPoint[1]+lslength] = False
        passed_data.append(np.array([originPoint[0],originPoint[1],lslength,lsdepth]))
        #print(slope_array_boolean.sum())
    return passed_data,failed_data,slope_array_boolean

# Function for placing landslides on an array the same shape as DEM.
def landslideBurner(landslideData,depth_array):
    for currentLandslide in landslideData:
        originPoint = [int(item) for item in currentLandslide[0:2]]
        lslength = int(currentLandslide[2])
        lsdepth = currentLandslide[3]
        depth_array[originPoint[0]:originPoint[0]+lslength, originPoint[1]:originPoint[1]+lslength] = lsdepth
    return depth_array

#Probability density function for making landslide distribution:
def pdf(x):
    return (1/(a*gamma(p)))*((a/(x-s))**(p+1))*np.exp(-1*a/(x-s))

def RasterMaker(array,output_file,reference_tiff,nodata):
    driver = gdal.GetDriverByName('GTiff')
    raster = driver.CreateCopy(output_file, reference_tiff, strict=0, options=["TILED=YES","COMPRESS=PACKBITS"])
    raster.GetRasterBand(1).WriteArray(array)
    raster.GetRasterBand(1).SetNoDataValue(nodata)
    raster = None

In [ ]:
if __name__ == '__main__':

    ################
    # User inputs: #
    ################

    #where input DEMs are saved
    folder = "/Users/home/fiemandi/CRNC/Topo_Analysis/"

    #DEM_name = 'Hapuku'
    #DEM_name = 'Kowhai'
    DEM_name = 'Baton'
    #How many landslide rasters would you like to make?
    N_landslides = 10

    #magnitude of event based on Malamud et al. (2004). USe if no idea how many landslides to create
    mag = 4
    #minimum landslide area (m^2)
    areals_min = 0.8 if DEM_name == 'Hapuku' else 8.1
    #maximum landslide area (m^2)
    areals_max = 527574.8#if DEM_name == 'Hapuku' else 165227.5
    #total number of landslides in the catchment
    #N_ls = 1554 if DEM_name == 'Hapuku' else 2823
    N_ls = 8000

    #Scalars for pdf and volume-area. See Malamud et al 2004
    a = 241.46 #scalar controlling lcoation of maximum probability
    s = -31.65 #scalar controlling exponential decay for small landslides
    p = 0.89 #scalar controlling power-law decay for medium and large landslides
    e = 0.08 #scalar used for volume (Malamud et al. 2004)
    y = 1.45 #exponent used for volume (Malamud et al. 2004)

    #minimum slope value where landslides can occur in degrees
    criticalSlope_degrees = 20.0

    #########
    # Main: #
    #########

    bins = math.isqrt(N_ls) #number of landslide areas/bins in the catchment
    ls_areas = np.geomspace(areals_min, areals_max, bins) #chosen landslide areas based on the max and min area and how many bins you want equal in logspace in m^2

    criticalSlope = math.tan(criticalSlope_degrees*(math.pi/180)) #convert slope degrees

    #Open slope array and make a landslide array
    slope_raster = gdal.Open(folder+DEM_name+'_r6_Slope.tif')
    nodata = slope_raster.GetRasterBand(1).GetNoDataValue()
    xres,yres = slope_raster.GetGeoTransform()[1:6:4]
    slope_array = slope_raster.GetRasterBand(1).ReadAsArray()
    print(f'opened {DEM_name} slope raster from DiffusionMaker')
    slope_array[slope_array==nodata]=np.nan

    #Getting the bin width of each landslide area:
    bin_width = []

    for i in range(0,(np.shape(ls_areas)[0]-1)):
        width = ls_areas[i+1]-ls_areas[i]
        bin_width.append(width)

    lsAreas = np.delete(ls_areas,0)#Each landslide area that will be plotted, uses the maximum area of each bin
    lsfreq = np.asarray(pdf(lsAreas)*N_ls*bin_width,dtype="int") #Number of landslides in each bin
    lslengths = np.sqrt(lsAreas) #Landslides are assummed to be a rectangle. Units in meters
    lsdepths = np.round((e*((np.array(lslengths)**2)**y))/(np.array(lslengths)**2),decimals=6) # meters - depth is based on the volume-area scaling in Malamud 2004

    # #For plotting pdf with landslide areas.
    # xdata = np.linspace(0, 3000000, num = 3000000)
    # fig,ax = plt.subplots(figsize=(8*(1/2.54),8*(1/2.54)),dpi=300)
    # ax.plot(xdata, pdf(xdata), color = 'indianred',label=r'$pdf(A_{ls,i}) = \dfrac{1}{a\Gamma(p)}\left(\dfrac{a}{A_{ls,i}-s}\right)^{p+1}e^{\frac{-a}{A_{ls,i}-s}}$')
    # ax.scatter(lsAreas, pdf(lsAreas), color = 'cornflowerblue',label='Landslide area chosen for model',s=10)
    # ax.set_xlabel('Landslide Area m$^{2}$')
    # ax.set_ylabel('Probability Density')
    # ax.set_yscale('log')
    # ax.set_xscale('log')
    # for ax in fig.get_axes():
        # ax.title.set_size(8)
        # ax.xaxis.label.set_size(8)
        # ax.yaxis.label.set_size(8)
        # ax.tick_params(axis='both',labelsize=6,direction='in',length=2,width=0.5)
        # for location in ['left', 'right', 'top', 'bottom']:
            # ax.spines[location].set_linewidth(0.5)
    # plt.legend(fontsize=7,loc='lower left')
    # plt.title(DEM_name,fontsize=9)
    # plt.tight_layout()
    # plt.savefig(DEM_name+'ModelPDF.png')
    # #plt.show()

In [ ]:
    #Get rid of landslides that have a frequency of zero
    maskfreq = np.where(lsfreq>0)
    ls_Freq = lsfreq[maskfreq]
    print('total number of landslides',np.sum(ls_Freq))
    print("Frequencies are:",ls_Freq)
    ls_Lengths = np.asarray((lslengths[maskfreq])/xres, dtype="int") #divide landslide length by resolution, units in cells
    print("Lengths are:",ls_Lengths)
    ls_Depths = lsdepths[maskfreq]
    #lsDepths = np.array([10.0,20.0,30.0])
    print("Depths are:",ls_Depths)


    N_landslide = 0

    while N_landslide < N_landslides:

        output_file = '/Users/home/fiemandi/CRNC/Landslide_Maker/Model_Landslides/'+f'{DEM_name}_ModelLandslides_{N_landslide}.tif'
        print('N_landslide:',N_landslide)

        #Initiate the empty landslide array
        landslideArray = np.full_like(slope_array,np.nan)

        #shuffle the lists so small landslides are not biased
        combined_list = list(zip(ls_Freq,ls_Lengths,ls_Depths))
        random.shuffle(combined_list)
        lsFreq,lsLengths,lsDepths = zip(*combined_list)
        lsFreq,lsLengths,lsDepths = list(lsFreq),list(lsLengths),list(lsDepths)

        print('Frequencies:',lsFreq)
        print('Lengths',lsLengths)
        print('Depths',lsDepths)

        #Filter throuh slope where values are only greater than critical slope value
        slopeRows,slopeCols = slopeFilter(slope_array, criticalSlope)

        #Create boolean array for slope
        slope_array_boolean = slope_array >= criticalSlope

        #Find indexes to put landslides
        #Outputs a list of data: [row, col, landslide length, landslide depth] for each landslide area for where it should be placed
        cpus = 40
        #print(len(slopeRows),len(slopeCols))
        start = datetime.datetime.now()
        with multiprocessing.Pool(processes=cpus) as cpu_pool:
            landslideDataList = cpu_pool.starmap(partial(landslideMasker,
                                                slopeDEM = slope_array_boolean, slopeRows = slopeRows, slopeCols = slopeCols),
                                                zip(lsLengths,lsDepths,lsFreq))
        end = datetime.datetime.now()

        landslideData = [item for sublist in landslideDataList for item in sublist]
        landslideData = np.array(landslideData)
        print('First landslideMasker took:',end-start)
        #"""
        #landslideData = np.load('./landslideDataList.npy')
        print('length of landslideData',len(landslideData))

        #Check to see if landslides overlap
        start = datetime.datetime.now()
        passedData,failedData,slope_array_boolean = landslideChecker(slope_array_boolean,landslideData)
        end = datetime.datetime.now()
        print("First landslideChecker completed in:",end-start)
        print("Passed:")
        print(len(passedData))
        print("Failed:")
        print(len(failedData))

        counter = 0
        while (len(failedData) > 0) and counter <= 2:
            temp_data = []
            start = datetime.datetime.now()
            failedLengths = [landslide[2] for landslide in failedData]
            failedDepths = [landslide[3] for landslide in failedData]

            #make frequencies from lengths
            failedLengths,failedFreq = np.unique(failedLengths, return_counts=True)
            failedDepths = np.unique(failedDepths)
            print('failed lengths:',failedLengths)
            print('failed depths:',failedDepths)
            print('failed freq:',failedFreq)

            if len(failedLengths) < 40:
                cpus = len(failedLengths)

            with multiprocessing.Pool(processes=cpus) as cpu_pool:
                landslideDataList = cpu_pool.starmap(partial(landslideMasker,
                                                     slopeDEM = slope_array_boolean, slopeRows = slopeRows, slopeCols = slopeCols),
                                                     zip(failedLengths.astype(int),failedDepths,failedFreq))
            #print(landslideDataList)
            landslideData = [item for sublist in landslideDataList for item in sublist]
            print(len(landslideData))
            passedData2,failedData,slope_array_boolean = landslideChecker(slope_array_boolean,np.array(landslideData))
            end = datetime.datetime.now()
            print('Next landslideMaskers took:',end-start)

            print("length of failed data:",len(failedData))
            print("length of passed data:",len(passedData2))
            passedData.extend(passedData2)
            counter += 1

        print('Number of tries placing failed landslides:',counter)

        print('length of failed data after all tries,', len(failedData))

        landslideArray = landslideBurner(passedData,landslideArray)
        print("Created landslide array with passed data")

        if len(failedData)==0:
            print("placed all landslides")
            RasterMaker(landslideArray,output_file,slope_raster, nodata)
            print('made landslide array into raster',output_file)

        if (len(failedData) > 0) and counter > 2:
            print('still failed landslides but not trying again sorry')
            RasterMaker(landslideArray,output_file,slope_raster, nodata)
            print('made landslide array into raster',output_file)

        N_landslide += 1

    sys.exit()